In [ ]:

!pip install streamlit==1.35.0 torchaudio==2.3.0 torch==2.3.0
!pip install git+https://github.com/R3gm/openvoice_package.git
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196

  Cloning https://github.com/R3gm/openvoice_package.git to /tmp/pip-req-build-ulqyjia3
  Running command git clone --filter=blob:none --quiet https://github.com/R3gm/openvoice_package.git /tmp/pip-req-build-ulqyjia3
  Resolved https://github.com/R3gm/openvoice_package.git to commit cbfefefce92ba515ef7e5769b589f93144cda04c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 83.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error origi

In [ ]:
import os
import requests
from PyPDF2 import PdfReader
from google.colab import drive

# ==============================
# CONFIG
# ==============================
DEEPGRAM_API_KEY = "c019dd7cefe1c3e7bbf2397f5a659dd252709877"
VOICE = "aura-asteria-en"   # Available Deepgram voices: aura-asteria-en, aura-thalia-en, etc.
OUTPUT_DIR = "/content/drive/MyDrive/audiobook"

# Mount Google Drive
drive.mount('/content/drive')

# Make sure output folder exists
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ==============================
# 1. Extract Text from PDF
# ==============================
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


# ==============================
# 2. Split Text into Chunks
# ==============================
def split_text(text, max_chars=2000):
    """
    Splits text into chunks that fit Deepgram’s API payload size.
    """
    words = text.split()
    chunks, chunk = [], ""
    for word in words:
        if len(chunk) + len(word) + 1 > max_chars:
            chunks.append(chunk)
            chunk = word
        else:
            chunk += " " + word
    if chunk:
        chunks.append(chunk.strip())
    return chunks


# ==============================
# 3. Convert Text to Speech
# ==============================
def text_to_speech(chunks, voice=VOICE):
    url = f"https://api.deepgram.com/v1/speak?model={voice}"
    headers = {
        "Authorization": f"Token {DEEPGRAM_API_KEY}",
        "Content-Type": "application/json"
    }

    for i, chunk in enumerate(chunks, start=1):
        output_file = os.path.join(OUTPUT_DIR, f"part_{i}.mp3")
        print(f"🔊 Converting chunk {i}/{len(chunks)} -> {output_file}")

        data = {"text": chunk}  # ✅ MUST specify ONLY text here
        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            with open(output_file, "wb") as f:
                f.write(response.content)
            print(f"✅ Saved {output_file}")
        else:
            print(f"❌ Error on chunk {i}: {response.text}")


# ==============================
# MAIN
# ==============================
pdf_path = "/content/drive/MyDrive/book.pdf"  # Change to your uploaded PDF file
text = extract_text_from_pdf(pdf_path)
chunks = split_text(text)

print(f"Total chunks: {len(chunks)}")
text_to_speech(chunks, voice=VOICE)
print("🎉 Audiobook generation complete!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total chunks: 369
🔊 Converting chunk 1/369 -> /content/drive/MyDrive/audiobook/part_1.mp3
✅ Saved /content/drive/MyDrive/audiobook/part_1.mp3
🔊 Converting chunk 2/369 -> /content/drive/MyDrive/audiobook/part_2.mp3
✅ Saved /content/drive/MyDrive/audiobook/part_2.mp3
🔊 Converting chunk 3/369 -> /content/drive/MyDrive/audiobook/part_3.mp3
✅ Saved /content/drive/MyDrive/audiobook/part_3.mp3
🔊 Converting chunk 4/369 -> /content/drive/MyDrive/audiobook/part_4.mp3
✅ Saved /content/drive/MyDrive/audiobook/part_4.mp3
🔊 Converting chunk 5/369 -> /content/drive/MyDrive/audiobook/part_5.mp3
✅ Saved /content/drive/MyDrive/audiobook/part_5.mp3
🔊 Converting chunk 6/369 -> /content/drive/MyDrive/audiobook/part_6.mp3
✅ Saved /content/drive/MyDrive/audiobook/part_6.mp3
🔊 Converting chunk 7/369 -> /content/drive/MyDrive/audiobook/part_7.mp3
✅ Saved /content/drive/MyDrive/audiob

In [ ]:
!mkdir -p checkpoints_v2
!wget -O checkpoints_v2/speaker_encoder.pth https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints_v2/speaker_encoder.pth
!wget -O checkpoints_v2/voice_conversion.pth https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints_v2/voice_conversion.pth


--2025-08-29 04:02:09--  https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints_v2/speaker_encoder.pth
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.49, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-08-29 04:02:09 ERROR 404: Not Found.

--2025-08-29 04:02:09--  https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints_v2/voice_conversion.pth
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.80, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-08-29 04:02:09 ERROR 404: Not Found.



In [ ]:
# ==============================
# STEP 0: Mount Google Drive
# ==============================
from google.colab import drive
drive.mount('/content/drive')

import os

# Update your paths here
REFERENCE_FILE = "/content/drive/MyDrive/voice_samples/myaudio2.wav"  # Your reference voice (5-10 sec, WAV)
SOURCE_FOLDER = "/content/drive/MyDrive/audiobook/"                    # Folder with TTS MP3 files
CONVERTED_FOLDER = "/content/drive/MyDrive/converted_audios/"          # Folder to save converted files
MODEL_FOLDER = "/content/drive/MyDrive/so-vits-svc/"                   # Folder to store pre-trained models

os.makedirs(CONVERTED_FOLDER, exist_ok=True)
os.makedirs(MODEL_FOLDER, exist_ok=True)

# ==============================
# STEP 1: Install Dependencies
# ==============================
!apt-get update
!apt-get install -y ffmpeg libsndfile1
!pip install git+https://github.com/voicepaw/so-vits-svc-fork.git
!pip install pydub

# ==============================
# STEP 2: Convert MP3s to WAV
# ==============================
from pydub import AudioSegment

source_files = [f for f in os.listdir(SOURCE_FOLDER) if f.lower().endswith(".mp3")]
for mp3_file in source_files:
    mp3_path = os.path.join(SOURCE_FOLDER, mp3_file)
    wav_path = os.path.join(CONVERTED_FOLDER, mp3_file.replace(".mp3", ".wav"))
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_frame_rate(44100).set_channels(1)
    audio.export(wav_path, format="wav")
    print(f"✅ Converted {mp3_file} → WAV")

# ==============================
# STEP 3: Download Pre-trained SO-VITS-SVC Models
# ==============================
# Make sure to create this folder structure: MODEL_FOLDER/logs/44k/
!mkdir -p "{MODEL_FOLDER}/logs/44k"

# Download generator and discriminator weights
!wget -O "{MODEL_FOLDER}/logs/44k/G_0.pth" "https://huggingface.co/cckavin/so-vits-svc/resolve/main/logs/44k/G_0.pth"
!wget -O "{MODEL_FOLDER}/logs/44k/D_0.pth" "https://huggingface.co/cckavin/so-vits-svc/resolve/main/logs/44k/D_0.pth"

# ==============================
# STEP 4: Batch Voice Conversion
# ==============================
from subprocess import run

for wav_file in os.listdir(CONVERTED_FOLDER):
    if wav_file.endswith(".wav"):
        input_path = os.path.join(CONVERTED_FOLDER, wav_file)
        output_path = os.path.join(CONVERTED_FOLDER, f"converted_{wav_file}")

        # Run SO-VITS-SVC inference
        run([
            "python3", "infer_tool.py",
            "--model_path", MODEL_FOLDER,
            "--input_audio", input_path,
            "--output_audio", output_path,
            "--reference_audio", REFERENCE_FILE
        ])
        print(f"✅ Converted {wav_file} → {output_path}")

# ==============================
# STEP 5: Listen to Converted Audio in Colab
# ==============================
from IPython.display import Audio, display

converted_files = [f for f in os.listdir(CONVERTED_FOLDER) if f.startswith("converted_")]
for file in converted_files:
    display(Audio(os.path.join(CONVERTED_FOLDER, file)))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages

In [ ]:
# ==============================
# Step 0: Install dependencies
# ==============================
!pip install -q pydub

from pydub import AudioSegment
import os

# ==============================
# Step 1: Set the folder containing .wav files
# ==============================
INPUT_FOLDER = "/content/drive/MyDrive/audiobook"  # Folder containing all 369 .wav files
OUTPUT_FILE = "/content/drive/MyDrive/merged_audio.wav"  # Path for final merged audio

# ==============================
# Step 2: List all .wav files and sort them
# ==============================
wav_files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(".wav")]
wav_files.sort()  # Sort alphabetically (adjust if needed)

# ==============================
# Step 3: Merge audios with short pause between them
# ==============================
if not wav_files:
    print("⚠️ No .wav files found in the folder.")
else:
    # Start with the first audio
    merged_audio = AudioSegment.from_wav(os.path.join(INPUT_FOLDER, wav_files[0]))
    pause = AudioSegment.silent(duration=500)  # 0.5 second silence between files

    # Append remaining audios
    for f in wav_files[1:]:
        audio = AudioSegment.from_wav(os.path.join(INPUT_FOLDER, f))
        merged_audio += pause + audio

    # Export final merged audio
    merged_audio.export(OUTPUT_FILE, format="wav")
    print(f"✅ Merged {len(wav_files)} audios into: {OUTPUT_FILE}")


✅ Merged 1 audios into: /content/drive/MyDrive/merged_audio.wav


In [ ]:
import os

with open("/content/drive/MyDrive/audiobook/parts.txt", "w") as f:
    for filename in sorted(os.listdir("/content/drive/MyDrive/audiobook")):
        if filename.startswith("part_") and filename.endswith(".mp3"):
            f.write(f"file '{filename}'\n")

print("✅ parts.txt created with the following content:\n")
print(open("/content/drive/MyDrive/audiobook/parts.txt").read())

✅ parts.txt created with the following content:

file 'part_1.mp3'
file 'part_10.mp3'
file 'part_100.mp3'
file 'part_101.mp3'
file 'part_102.mp3'
file 'part_103.mp3'
file 'part_104.mp3'
file 'part_105.mp3'
file 'part_106.mp3'
file 'part_107.mp3'
file 'part_108.mp3'
file 'part_109.mp3'
file 'part_11.mp3'
file 'part_110.mp3'
file 'part_111.mp3'
file 'part_112.mp3'
file 'part_113.mp3'
file 'part_114.mp3'
file 'part_115.mp3'
file 'part_116.mp3'
file 'part_117.mp3'
file 'part_118.mp3'
file 'part_119.mp3'
file 'part_12.mp3'
file 'part_120.mp3'
file 'part_121.mp3'
file 'part_122.mp3'
file 'part_123.mp3'
file 'part_124.mp3'
file 'part_125.mp3'
file 'part_126.mp3'
file 'part_127.mp3'
file 'part_128.mp3'
file 'part_129.mp3'
file 'part_13.mp3'
file 'part_130.mp3'
file 'part_131.mp3'
file 'part_132.mp3'
file 'part_133.mp3'
file 'part_134.mp3'
file 'part_135.mp3'
file 'part_136.mp3'
file 'part_137.mp3'
file 'part_138.mp3'
file 'part_139.mp3'
file 'part_14.mp3'
file 'part_140.mp3'
file 'part_141.mp3

In [ ]:
!ffmpeg -f concat -safe 0 -i /content/drive/MyDrive/audiobook/parts.txt -c copy /content/drive/MyDrive/audiobook/life3_ai.mp3

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab